<a href="https://colab.research.google.com/github/geetha2612/deep__learning/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku
import tensorflow

# set seeds for reproducability
tensorflow.random.set_seed(200)

import pandas as pd
import numpy as np
import string

In [ ]:
all_headlines = []
article_df = pd.read_csv('/content/ArticlesMarch2018.csv')
all_headlines.extend(list(article_df.headline.values))
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

1250

In [ ]:
all_headlines[:10]
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['virtual coins real resources',
 'us advances military plans for north korea',
 'mr trump and the very bad judge',
 'to erase dissent china bans pooh bear and n',
 'loans flowed to kushner cos after visits to the white house',
 'china envoy intends to ease trade tensions',
 'president trumps contradictory and sometimes false comments about gun policy to lawmakers',
 'classic letter puzzle',
 'silicon valley disruption in an australian school',
 'the assassination of gianni versace episode 6 a nothing man']

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
token_list = tokenizer.texts_to_sequences(["I am happy to see you here today"])[0]
print(token_list)

check=[]

for i in range(1, len(token_list)):
  n_gram_sequence = token_list[:i+1]
  check.append(n_gram_sequence)

check

[33, 469, 1062, 3, 191, 16, 84, 767]


[[33, 469],
 [33, 469, 1062],
 [33, 469, 1062, 3],
 [33, 469, 1062, 3, 191],
 [33, 469, 1062, 3, 191, 16],
 [33, 469, 1062, 3, 191, 16, 84],
 [33, 469, 1062, 3, 191, 16, 84, 767]]

In [ ]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    ## convert data to sequence of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)

inp_sequences[:10], total_words

([[1119, 1120],
  [1119, 1120, 116],
  [1119, 1120, 116, 1121],
  [31, 1122],
  [31, 1122, 589],
  [31, 1122, 589, 392],
  [31, 1122, 589, 392, 7],
  [31, 1122, 589, 392, 7, 61],
  [31, 1122, 589, 392, 7, 61, 70],
  [117, 10]],
 3582)

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
predictors,label,len(label[0]),max_sequence_len

(array([[   0,    0,    0, ...,    0,    0, 1119],
        [   0,    0,    0, ...,    0, 1119, 1120],
        [   0,    0,    0, ..., 1119, 1120,  116],
        ...,
        [   0,    0,    0, ...,  979,  151,  386],
        [   0,    0,    0, ...,    0,    0, 3581],
        [   0,    0,    0, ...,    0, 3581,    5]], dtype=int32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 3582,
 18)

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()

    # Setting early_stopping feature to stop early on getting stagnant
    early_stopping = EarlyStopping(
    min_delta=0.01, # minimium amount of change to count as an improvement
    patience=5, # how many epochs to wait before stopping
    restore_best_weights=True,
    )

    # Add Input Embedding Layer
    model.add(Embedding(total_words, 32, input_length=input_len))

    # input_dim: Integer. Size of the vocabulary, i.e. maximum integer index + 1.
    # output_dim: Integer. Dimension of the dense embedding.
    # input_length: Length of input sequences, when it is constant.

    # Add Hidden Layer 1 - RNN Layer
    model.add(SimpleRNN(200))

    # Add Hidden Layer 2 - Dropout Layer
    model.add(Dropout(0.1))

    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 17, 32)            114624    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 200)               46600     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 3582)              719982    
                                                                 
Total params: 881206 (3.36 MB)
Trainable params: 881206 (3.36 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(predictors, label, epochs=100)


Epoch 1/100
252/252 [==============================] - 7s 28ms/step - loss: 0.4757
Epoch 2/100
252/252 [==============================] - 8s 33ms/step - loss: 0.4585
Epoch 3/100
252/252 [==============================] - 8s 30ms/step - loss: 0.4417
Epoch 4/100
252/252 [==============================] - 8s 33ms/step - loss: 0.4126
Epoch 5/100
252/252 [==============================] - 9s 34ms/step - loss: 0.4082
Epoch 6/100
252/252 [==============================] - 7s 28ms/step - loss: 0.4044
Epoch 7/100
252/252 [==============================] - 8s 32ms/step - loss: 0.3928
Epoch 8/100
252/252 [==============================] - 8s 30ms/step - loss: 0.3951
Epoch 9/100
252/252 [==============================] - 8s 31ms/step - loss: 0.3885
Epoch 10/100
252/252 [==============================] - 9s 34ms/step - loss: 0.3859
Epoch 11/100
252/252 [==============================] - 7s 29ms/step - loss: 0.3666
Epoch 12/100
252/252 [==============================] - 8s 33ms/step - loss: 0.3699
E

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probabilities = model.predict(token_list)[0]
        # Sample the next word based on predicted probabilities
        predicted_index = np.random.choice(len(predicted_probabilities), p=predicted_probabilities)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [ ]:
print (generate_text("jack ", 5, model, max_sequence_len))
print (generate_text("president trump",5, model, max_sequence_len))
print (generate_text("donald", 6, model, max_sequence_len))
print (generate_text("india and china", 2, model, max_sequence_len))
print (generate_text("new york", 4, model, max_sequence_len))
print (generate_text("science and technology", 6, model, max_sequence_len))

1/1 [==============================] - 0s 44ms/step
jack  the simple serendipitous joy of
1/1 [==============================] - 0s 31ms/step
president trump more you have about years
1/1 [==============================] - 0s 36ms/step
donald trumps tunnel vision difference that comments
1/1 [==============================] - 0s 38ms/step
india and china plans for
1/1 [==============================] - 0s 29ms/step
new york has 7 billion reasons
1/1 [==============================] - 0s 26ms/step
science and technology what to watch for with job
